## Entrenamiento del modelo y predicciones en test
Aquí se entrena el modelo con todas las facturas disponibles, de cara a usarlo para el reto.

In [ ]:
!pip install transformers
!pip install datasets
!pip install torch
!pip install seqeval
!pip install accelerate -U
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict, Features, Sequence, ClassLabel, Value
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification, logging
from transformers import __version__ as transformers_version
import numpy as np
from sklearn.model_selection import train_test_split
import seqeval.metrics
from google.colab import files
from google.colab import drive
import evaluate
import numpy as np


# Configurar el nivel de verbosidad del logging
logging.set_verbosity_error()


In [ ]:
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
csv_path = "/content/drive/MyDrive/dataset_facturas_aumentado_etiquetado.csv"
df = pd.read_csv(csv_path)

# Verificar que se ha leído correctamente
print("Las primeras filas del DataFrame:")
df.head(3)


Las primeras filas del DataFrame:


,id,texto,ner_tags,json,original,json_comprobacion,json_2
0,factura_836,"['DATOS', 'DE', 'LA', 'FACTURA', 'IMPORTE', 'F...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-FAC', '...","{'nombre_cliente': 'ERMINIA CUESTA MACHADO', '...",1,"{'nombre_cliente': ['ERMINIA CUESTA MACHADO', ...","{'nombre_cliente': 'ERMINIA CUESTA MACHADO', '..."
1,factura_264,"['L', '..', '.', 'u', '.', '.', 'n', '..e', '....","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","{'nombre_cliente': 'SALUD MONTALVO CUÉLLAR', '...",1,"{'nombre_cliente': ['SALUD MONTALVO CUÉLLAR', ...","{'nombre_cliente': 'SALUD MONTALVO CUÉLLAR', '..."
2,factura_533,"['DATOS', 'DE', 'LA', 'FACTURA', 'Nº', 'factur...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NUMF', ...","{'nombre_cliente': 'Iluminada Donato Vallejo',...",1,{'nombre_cliente': ['Iluminada Donato Vallejo'...,"{'nombre_cliente': 'Iluminada Donato Vallejo',..."


Lo primero que tenemos que hacer es convertir el datafreame en un objeto datasets de Hugingface



In [ ]:
# Seleccionar solo las columnas 'id', 'texto' y 'ner_tags'
df_subset = df.loc[:, ['id', 'texto', 'ner_tags']]

# Renombrar la columna 'texto' a 'tokens'
df_subset.rename(columns={'texto': 'tokens'}, inplace=True)

# Mostrar las primeras filas del nuevo DataFrame para verificar
df_subset.head(3)

,id,tokens,ner_tags
0,factura_836,"['DATOS', 'DE', 'LA', 'FACTURA', 'IMPORTE', 'F...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-FAC', '..."
1,factura_264,"['L', '..', '.', 'u', '.', '.', 'n', '..e', '....","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,factura_533,"['DATOS', 'DE', 'LA', 'FACTURA', 'Nº', 'factur...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NUMF', ..."


In [ ]:
# Definir una función para convertir el contenido de una columna en una lista
def to_list(column):
    # Eliminar los corchetes adicionales y convertir la cadena en lista
    return eval(column)

# Aplicar la función a las columnas 'tokens' y 'ner_tags'
df_subset['tokens'] = df_subset['tokens'].apply(to_list)
df_subset['ner_tags'] = df_subset['ner_tags'].apply(to_list)

In [ ]:
# Dividir df_subset en conjuntos de entrenamiento y validación
train, validation = train_test_split(df_subset, test_size=0.2, random_state=42)

# Imprimir el número de filas en los conjuntos de entrenamiento y prueba
print("Número de filas en el conjunto de entrenamiento:", len(train))
print("Número de filas en el conjunto de validación:", len(validation))

Número de filas en el conjunto de entrenamiento: 1462
Número de filas en el conjunto de validación: 366


In [ ]:
# Definir la lista de etiquetas
label_list = ['O', 'B-NOM', 'I-NOM', 'B-DNI', 'I-DNI', 'B-CAL', 'I-CAL', 'B-CP', 'I-CP', 'B-LOC', 'I-LOC', 'B-PRO', 'I-PRO',
            'B-NOMC', 'I-NOMC', 'B-CIF', 'I-CIF', 'B-DIRC', 'I-DIRC', 'B-CPC', 'I-CPC', 'B-LOCC', 'I-LOCC', 'B-PROC', 'I-PROC',
            'B-NUMF', 'I-NUMF', 'B-INI', 'I-INI', 'B-FIN', 'I-FIN', 'B-FAC', 'I-FAC', 'B-CAR','I-CAR', 'B-PER',
            'I-PER', 'B-POT', 'I-POT']
label_to_id = {label: i for i, label in enumerate(label_list)}

num_labels = len(label_list)

In [ ]:
# Función para convertir etiquetas de texto a etiquetas numéricas
def convert_labels_to_ids(labels, label_to_id):
    return [[label_to_id[label] for label in label_seq] for label_seq in labels]

# Convertir las etiquetas en cada DataFrame
def process_dataframe(df, label_to_id):
    df['ner_tags'] = convert_labels_to_ids(df['ner_tags'], label_to_id)
    return df


validation = process_dataframe(validation, label_to_id)
train = process_dataframe(train, label_to_id)

In [ ]:
# Convertir DataFrames a Datasets de Hugging Face
train_dataset = Dataset.from_pandas(train)
validation_dataset = Dataset.from_pandas(validation)

# Eliminar la columna '__index_level_0__' si existe
train_dataset = train_dataset.remove_columns(['__index_level_0__'])
validation_dataset = validation_dataset.remove_columns(['__index_level_0__'])

# Definir las características del dataset
features = Features({
    'id': Value(dtype='string'),
    'tokens': Sequence(Value(dtype='string')),
    'ner_tags': Sequence(ClassLabel(names=label_list))
})

# Asignar las características a cada dataset
train_dataset = train_dataset.cast(features)
validation_dataset = validation_dataset.cast(features)

# Crear el DatasetDict
datasets = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset
})

print(datasets)


Casting the dataset:   0%|          | 0/1462 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/366 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1462
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 366
    })
})


In [ ]:
datasets['train'].features['ner_tags']

Sequence(feature=ClassLabel(names=['O', 'B-NOM', 'I-NOM', 'B-DNI', 'I-DNI', 'B-CAL', 'I-CAL', 'B-CP', 'I-CP', 'B-LOC', 'I-LOC', 'B-PRO', 'I-PRO', 'B-NOMC', 'I-NOMC', 'B-CIF', 'I-CIF', 'B-DIRC', 'I-DIRC', 'B-CPC', 'I-CPC', 'B-LOCC', 'I-LOCC', 'B-PROC', 'I-PROC', 'B-NUMF', 'I-NUMF', 'B-INI', 'I-INI', 'B-FIN', 'I-FIN', 'B-FAC', 'I-FAC', 'B-CAR', 'I-CAR', 'B-PER', 'I-PER', 'B-POT', 'I-POT'], id=None), length=-1, id=None)

El modelo que vamos a finetunear es: 'dccuchile/bert-base-spanish-wwm-cased', conocido coloquialmente como BERTO.



In [ ]:
task = "ner"
#model_checkpoint = "mrm8488/distilbert-base-multi-cased-finetuned-typo-detection"
model_checkpoint =  'dccuchile/bert-base-spanish-wwm-cased'
batch_size = 16

Una vez creado el objeto datasets y cargado el modelo, procedemos a tokenizar el texto y alinear las etiquetas NER para que siga habiendo correspondencia.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

In [ ]:
label_all_tokens = True

def tokenize_and_align_labels(examples):
    # Tokenizar las entradas con truncamiento
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True, max_length=512)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Los tokens especiales tienen un ID de palabra que es None. Se establece la etiqueta en -100 para que se ignoren en la función de pérdida.
            if word_idx is None:
                label_ids.append(-100)
            # Se establece la etiqueta para el primer token de cada palabra.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # Para los otros tokens en una palabra, se establece la etiqueta en la etiqueta actual o -100, dependiendo del flag label_all_tokens.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/1462 [00:00<?, ? examples/s]

Map:   0%|          | 0/366 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1462
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 366
    })
})

Ahora procedemos a entrenar y evaluar el modelo.






In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=num_labels,
    ignore_mismatched_sizes=True
)

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,
    learning_rate= 2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=15,
    weight_decay=0.35,
    push_to_hub=False,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Definir el data collator
data_collator = DataCollatorForTokenClassification(tokenizer)


In [ ]:
# Cargar la métrica utilizando evaluate
metric = evaluate.load("seqeval")


In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()


{'loss': 1.7113, 'grad_norm': 1.4296518564224243, 'learning_rate': 1.9855072463768115e-05, 'epoch': 0.10869565217391304}
{'loss': 0.7983, 'grad_norm': 1.1246832609176636, 'learning_rate': 1.9710144927536236e-05, 'epoch': 0.21739130434782608}
{'loss': 0.6165, 'grad_norm': 1.0046850442886353, 'learning_rate': 1.956521739130435e-05, 'epoch': 0.32608695652173914}
{'loss': 0.4142, 'grad_norm': 0.7234253883361816, 'learning_rate': 1.9420289855072467e-05, 'epoch': 0.43478260869565216}
{'loss': 0.3069, 'grad_norm': 1.001856803894043, 'learning_rate': 1.927536231884058e-05, 'epoch': 0.5434782608695652}
{'loss': 0.2235, 'grad_norm': 0.8242420554161072, 'learning_rate': 1.9130434782608697e-05, 'epoch': 0.6521739130434783}
{'loss': 0.1795, 'grad_norm': 0.7188001871109009, 'learning_rate': 1.8985507246376814e-05, 'epoch': 0.7608695652173914}
{'loss': 0.1321, 'grad_norm': 0.6954039335250854, 'learning_rate': 1.8840579710144928e-05, 'epoch': 0.8695652173913043}
{'loss': 0.1197, 'grad_norm': 0.6166937

TrainOutput(global_step=1380, training_loss=0.04683966284779751, metrics={'train_runtime': 2292.726, 'train_samples_per_second': 9.565, 'train_steps_per_second': 0.602, 'train_loss': 0.04683966284779751, 'epoch': 15.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.03511938080191612, 'eval_precision': 0.9450599582898853, 'eval_recall': 0.9639699528019677, 'eval_f1': 0.9544212985816304, 'eval_accuracy': 0.9920497160612879, 'eval_runtime': 14.2682, 'eval_samples_per_second': 25.651, 'eval_steps_per_second': 1.612, 'epoch': 15.0}


{'eval_loss': 0.03511938080191612,
 'eval_precision': 0.9450599582898853,
 'eval_recall': 0.9639699528019677,
 'eval_f1': 0.9544212985816304,
 'eval_accuracy': 0.9920497160612879,
 'eval_runtime': 14.2682,
 'eval_samples_per_second': 25.651,
 'eval_steps_per_second': 1.612,
 'epoch': 15.0}

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'CAL': {'precision': 0.9863325740318907,
  'recall': 0.9885844748858448,
  'f1': 0.9874572405929305,
  'number': 438},
 'CAR': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 409},
 'CIF': {'precision': 0.9940387481371088,
  'recall': 0.9866863905325444,
  'f1': 0.9903489235337788,
  'number': 1352},
 'CP': {'precision': 0.9895397489539749,
  'recall': 0.9957894736842106,
  'f1': 0.9926547743966422,
  'number': 950},
 'CPC': {'precision': 0.9819059107358263,
  'recall': 0.996328029375765,
  'f1': 0.9890643985419199,
  'number': 817},
 'DIRC': {'precision': 0.9347826086956522,
  'recall': 0.9707835325365206,
  'f1': 0.952442996742671,
  'number': 753},
 'DNI': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 645},
 'FAC': {'precision': 0.5564516129032258,
  'recall': 0.777027027027027,
  'f1': 0.6484962406015038,
  'number': 444},
 'FIN': {'precision': 0.9267271458478716,
  'recall': 0.9405099150141643,
  'f1': 0.9335676625659051,
  'number': 1412},
 'INI': {'precision': 0

In [ ]:
trainer.save_model("/content/drive/MyDrive/model_15_035_def")

Si bien las métricas pueden parecer muy buenas, al haber tantas clases y estar estas tan desbalanceadas, el resultado es engañoso.